# Data Ingest of 10-h Fuel Moisture Content

This notebook demonstrates retrieval and filtering of 10-h dead FMC data from RAWS. Retrieval of 10-h FMC observations is done with the software package `SynopticPy` and a stash of RAWS data kept and maintained by the broader OpenWFM community. This notebook will demonstrate use of `Synopticpy` with a free token, so limits are placed on the number of sensor hours that can be requested.

For more info, see Brian Blaylock's `SynopticPy` [python package](https://github.com/blaylockbk/SynopticPy)

The main steps in the retrieval are:
* Use `synoptic.Metadata` to determine the RAWS with FMC data in the given spatial domain and time frame
* 

## Setup

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import synoptic
import json
import sys
sys.path.append('../src')
from utils import Dict

A configuration file is used to control data ingest. Automated processes utilize the file `training_data_config.json` or `forecast_config.json`. In this tutorial, we will manually build a config file

In [ ]:
config = Dict({
    'start_time': '2024-01-01_00:00:00',
    'end_time': '2024-01-01_02:00:00',
    'bbox': [37, -111, 46, -95],
    'raws_vars': ["fuel_moisture"]
})

config

## Stations MetaData

*Note*: the bounding box format used in `wrfxpy` is `[min_lat, min_lon, max_lat, max_lon]`. But, the bounding box format used by Synoptic is `[min_lon, min_lat, max_lon, max_lat]`.

In [ ]:
bbox = config.bbox
bbox_reordered = [bbox[1], bbox[0], bbox[3], bbox[2]]
start = datetime.strptime(config.start_time, "%Y-%m-%d_%H:%M:%S")
end = datetime.strptime(config.end_time, "%Y-%m-%d_%H:%M:%S")
raws_vars = config.raws_vars

In [ ]:
sts = synoptic.Metadata(
    bbox=bbox_reordered,
    vars=raws_vars,
    obrange=(start, end),
).df()

In [ ]:
sts

## Station Time Series

We loop over the station IDs found in the previous step and retrieve all available data.

*NOTE*: this process is not parallelized, as the same IP address is used for each request and parallization may result in issues

In [ ]:
print(f"Attempting retrieval of RAWS from {start} to {end} within {bbox}")
print("~"*75)

raws_dict = {}

for st in sts['stid']:
    print("~"*50)
    print(f"Attempting retrival of station {st}")
    df_temp = synoptic.TimeSeries(
        stid="CPTC2",
        start=start,
        end=end,
        vars=["fuel_moisture"]
    ).df()
    if df_temp.shape[0] > 0:
        print(f"Found {df_temp.shape[0]} FMC records. Saving to data dictionary")
        raws_dict[st] = df_temp

In [ ]:
raws_dict.keys()

In [ ]:
raws_dict['CPTC2']